<a href="https://colab.research.google.com/github/seungsdew/-/blob/main/Lecture04_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:

import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision.transforms as T

from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
from PIL import Image

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

MNIST_ROOT = "/content/MNIST"
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/MNIST

!unzip -qq "/content/drive/MyDrive/MNIST.zip"

[Errno 2] No such file or directory: '/content/drive/MyDrive/MNIST'
/content
replace train/0/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [14]:
# 데이터셋 구조도
for phase in os.listdir(MNIST_ROOT):
    print(MNIST_ROOT+"/"+phase)
    for dir in os.listdir(os.path.join(MNIST_ROOT, phase)):
        print(f"---------------------/", end="")
        print(dir, f"number : {len(os.listdir(os.path.join(MNIST_ROOT, phase, dir)))}")

/content/MNIST/val
---------------------/6 number : 958
---------------------/7 number : 1028
---------------------/5 number : 892
---------------------/0 number : 980
---------------------/1 number : 1135
---------------------/3 number : 1010
---------------------/9 number : 1009
---------------------/8 number : 974
---------------------/2 number : 1032
---------------------/4 number : 982
/content/MNIST/train
---------------------/6 number : 5918
---------------------/7 number : 6265
---------------------/5 number : 5421
---------------------/0 number : 5923
---------------------/1 number : 6742
---------------------/3 number : 6131
---------------------/9 number : 5949
---------------------/8 number : 5851
---------------------/2 number : 5958
---------------------/4 number : 5842


# torch.utils.data.Dataset 소개

PyTorch에서 `torch.utils.data.Dataset`은 데이터셋을 나타내는 클래스입니다. 이 클래스는 데이터에 대한 접근과 조작을 효율적으로 수행할 수 있는 인터페이스를 제공합니다. `Dataset` 클래스는 데이터와 모델 사이의 연결고리 역할을 합니다.

## 주요 기능
1. **데이터 로딩**: `Dataset`은 파일, 데이터베이스, API 등 다양한 소스에서 데이터를 편리하게 로딩할 수 있는 방법을 제공합니다. 이를 통해 특정 요구에 맞게 데이터 로딩 과정을 사용자 정의할 수 있습니다.

2. **데이터 변환**: `Dataset`은 데이터에 전처리, 데이터 증강, 특성 엔지니어링 등의 변환을 실시간으로 적용할 수 있습니다. 이러한 변환은 데이터 로딩 파이프라인에 원활하게 통합될 수 있습니다.

3. **데이터 접근**: `Dataset`은 색인을 사용하여 개별 데이터 샘플에 효율적으로 접근할 수 있는 방법을 제공합니다. 데이터에 대해 무작위 또는 순차적인 접근을 허용하므로 배치 처리 및 단일 샘플 처리에 모두 적합합니다.

## 커스텀 데이터셋 생성하기
`torch.utils.data.Dataset`을 사용하기 위해선, `Dataset` 클래스를 상속받고 두 가지 중요한 메서드를 구현하는 방식으로 커스텀 데이터셋을 생성할 수 있습니다:

1. `__len__()`: 이 메서드는 데이터셋의 크기, 즉 총 샘플 수를 반환합니다.

2. `__getitem__(index)`: 이 메서드는 주어진 인덱스에 해당하는 샘플을 반환합니다. 데이터를 로딩하고 필요한 변환을 적용합니다.

In [15]:
# 이미지를 읽는 함수 정의
def get_image(p: str):
    return Image.open(p).convert("L")

# 데이터셋 클래스 정의
class baseDataset(Dataset):
    def __init__(self, root: str, train: bool):
        super().__init__()

        # 훈련 데이터셋 또는 검증 데이터셋의 루트 디렉토리 설정
        if train:
            self.root = os.path.join(root, "train")
            self.transform = T.Compose([
                T.ToTensor()
            ])
        else:
            self.root = os.path.join(root, "val")
            self.transform = T.Compose([
                T.ToTensor()
            ])

        # 데이터 리스트 생성
        data_list = []
        for i in range(10):
            dir = os.path.join(self.root, str(i))
            for img in os.listdir(dir):
                img_path = os.path.join(dir, img)
                data_list.append((i, img_path))
        self.data_list = data_list

    def __len__(self):
        # 데이터셋의 총 데이터 개수 반환
        return len(self.data_list)

    def __getitem__(self, idx: int):
        # 인덱스에 해당하는 데이터 반환
        number, img_path = self.data_list[idx]

        # 이미지 파일을 PIL 객체로 읽어들이고, 그레이스케일로 변환한 후 텐서로 변환
        img_obj = get_image(img_path)
        img_tensor = self.transform(img_obj)

        return img_tensor, number

# baseDataset 테스트
test_dataset = baseDataset(MNIST_ROOT, False)  # 검증 데이터셋 객체 생성
print(f"length = {len(test_dataset)}")  # 데이터셋의 총 데이터 개수 출력

sample_tensor, sample_target = test_dataset[10]  # 인덱스 10에 해당하는 데이터 샘플 가져오기
C, H, W = sample_tensor.shape  # 텐서의 채널, 높이, 너비 출력
print(f"Channel = {C}, Height = {H}, Width = {W}")
print(f"sample_target = {type(sample_target)}, {sample_target}")

length = 10000
Channel = 1, Height = 28, Width = 28
sample_target = <class 'int'>, 0


# Multi-Layer Perceptron (다층 퍼셉트론)

Multi-Layer Perceptron(MLP)은 인공신경망의 가장 기본적인 형태로, 여러 개의 은닉층(hidden layer)으로 구성된 다층 구조의 신경망입니다. MLP는 입력층(input layer), 은닉층(hidden layer), 출력층(output layer)으로 구성되며, 각 층은 여러 개의 뉴런으로 구성됩니다.

주요 특징과 동작 원리는 다음과 같습니다:

- **다층 구조**: MLP는 여러 개의 은닉층을 가지는 다층 구조로 이루어져 있습니다. 은닉층은 입력층과 출력층 사이에 위치하며, 입력층의 정보를 받아 처리한 후 출력층으로 전달합니다.

- **비선형성 적용**: MLP는 각 은닉층과 출력층에 비선형 활성화 함수를 적용합니다. 이 비선형성은 MLP가 복잡한 함수를 모델링할 수 있도록 하며, 선형 모델로는 표현하기 어려운 비선형 관계를 학습할 수 있게 합니다.

- **전방향 전파**: MLP는 입력층에서부터 은닉층을 거쳐 출력층으로 데이터를 전달하는 전방향 전파(forward propagation)를 수행합니다. 입력 데이터는 각 은닉층을 통과하면서 가중치와 편향에 의해 변환되고, 활성화 함수를 통과하여 출력층으로 전달됩니다.

- **가중치 업데이트**: MLP는 역전파 알고리즘을 사용하여 가중치를 업데이트합니다. 역전파 알고리즘은 손실 함수의 그래디언트를 계산하고, 이를 이용하여 가중치를 조정하여 손실을 최소화하는 방향으로 모델을 학습시킵니다.

- **다중 분류와 회귀**: MLP는 다중 클래스 분류와 회귀 문제에 모두 사용될 수 있습니다. 다중 클래스 분류에서는 출력층의 뉴런 수가 클래스 수와 일치하며, 각 뉴런은 해당 클래스에 대한 확률을 나타냅니다. 회귀 문제에서는 출력층의 뉴런 수가 1이며, 예측한 값을 출력합니다.

- **학습과 일반화**: MLP는 주어진 입력과 정답(label)을 사용하여 가중치를 학습시킵니다. 학습된

 MLP는 새로운 입력에 대한 예측을 수행할 수 있으며, 학습 데이터에 대한 정확도를 넘어서 일반화(generalization)된 예측을 수행합니다.

MLP는 다층 구조와 비선형 활성화 함수를 통해 복잡한 함수를 모델링할 수 있는 강력한 신경망입니다. 이를 통해 이미지 분류, 텍스트 분류, 회귀 등 다양한 머신러닝과 딥러닝 문제를 해결할 수 있습니다.

# torch.nn.Module 소개

`torch.nn.Module`은 PyTorch에서 신경망 모델을 구성하는 데 사용되는 핵심 클래스입니다. 이 클래스를 활용하여 다양한 신경망 계층을 정의하고 연결하여 모델을 구축할 수 있습니다.

## 주요 기능
1. **신경망 계층 정의**: `Module`을 상속하여 사용자 정의 신경망 계층을 구현할 수 있습니다. 각 계층은 `forward()` 메서드를 통해 입력을 받고 출력을 반환합니다.

2. **파라미터 관리**: `Module`은 자체적으로 파라미터를 관리합니다. 각 계층의 학습 가능한 가중치와 편향은 모델 내부에서 자동으로 추적되며, 최적화 과정에서 업데이트됩니다.

3. **순전파 (Forward Propagation)**: `Module`을 통해 정의한 신경망은 순전파를 통해 입력 데이터를 출력으로 변환합니다. `forward()` 메서드에서 계층을 연결하고 데이터를 처리하여 출력을 생성합니다.


# nn.Sequential 소개

`nn.Sequential`은 PyTorch에서 신경망 모델을 구성하기 위해 사용되는 클래스입니다. 이 클래스는 여러 개의 신경망 계층을 연속적으로 연결하여 모델을 간단하게 정의할 수 있는 도구입니다.

## 주요 특징
1. **계층 순차적 구성**: `nn.Sequential`은 순차적으로 여러 개의 신경망 계층을 정의할 수 있습니다. 계층은 순서대로 연결되어 입력 데이터를 처리하고 출력을 생성합니다.

2. **간편한 모델 정의**: `nn.Sequential`은 직관적이고 간단한 방식으로 모델을 정의할 수 있습니다. 계층을 리스트 또는 OrderedDict 형태로 전달하여 모델을 구성할 수 있습니다.

3. **자동적인 순전파**: `nn.Sequential`은 입력 데이터를 전달하면 내부적으로 순전파를 자동으로 수행합니다. 계층을 차례대로 통과하여 출력을 생성하므로 개별 계층을 직접 호출할 필요가 없습니다.

## 사용법
`nn.Sequential`을 사용하기 위해서는 계층을 차례대로 나열하여 전달해야 합니다. 아래는 `nn.Sequential`을 사용하여 모델을 정의하는 예시입니다:

```python
import torch.nn as nn

# 모델 정의
model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, output_size),
    nn.Softmax(dim=1)
)
```

위 예시에서 `nn.Sequential`은 입력 데이터를 받아 각 계층을 차례로 통과시키고 최종 출력을 생성하는 모델을 정의합니다. 각 계층은 리스트 안에 순서대로 나열됩니다.


# Activation Function의 역할

Activation Function(활성화 함수)은 Deep Neural Network(DNN)에서 중요한 역할을 수행합니다. Activation Function은 신경망의 각 뉴런에 적용되며, 입력 신호를 비선형적으로 변환하여 네트워크의 표현력을 향상시킵니다. 아래는 Activation Function의 주요 역할에 대한 설명입니다:

1. **비선형성 표현**: Activation Function은 신경망에 비선형성을 주어 선형 모델로는 표현할 수 없는 복잡한 함수를 학습할 수 있도록 합니다. 비선형성은 신경망이 다양한 종류의 데이터 패턴을 학습하고 복잡한 결정 경계를 생성할 수 있도록 돕습니다.

2. **신경망의 표현력 증가**: Activation Function은 신경망이 다양한 함수를 근사할 수 있도록 합니다. 비선형 함수를 통과한 출력은 다음 레이어로 전달되며, 이를 통해 네트워크는 더 다양하고 복잡한 함수를 모델링할 수 있게 됩니다.

3. **Gradient의 전파**: Activation Function은 역전파(Backpropagation) 과정에서 그래디언트(Gradient)를 적절하게 전달할 수 있도록 합니다. 비선형 함수의 도함수를 계산하고, 이를 통해 이전 레이어로 그래디언트를 역전파하여 학습을 진행할 수 있습니다.

4. **출력의 정규화**: Activation Function은 출력값의 범위를 제한하여 모델의 안정성을 향상시킵니다. 예를 들어, Sigmoid Activation Function은 출력값을 0과 1 사이로 제한하며, Hyperbolic Tangent (tanh) Activation Function은 -1과 1 사이로 제한합니다. 이를 통해 네트워크의 출력을 제한하고, 입력에 대한 모델의 민감도를 조절할 수 있습니다.

일반적으로 사용되는 Activation Function으로는 Sigmoid, ReLU, tanh, Leaky ReLU 등이 있습니다. 이들 Activation Function은 각자의 특성과 장단점을 가지고 있으며, 문제에 맞게 선택되어 사용됩니다. Activation Function의 역할은 DNN의 성능과 수렴 속도에 큰 영향을 미치므로, 적절한 Activation Function의 선택은 모델의 효과적인 학습과 성능 향상에 중요합니다.

Image Classification 모델에서 마지막 레이어에 Activation 함수를 사용하지 않는 이유는 다음과 같습니다:

1. **Output Space의 제약**: 일반적으로 Image Classification 모델의 마지막 레이어는 클래스의 확률을 나타내기 위한 선형 레이어로 구성됩니다. 이 선형 레이어는 출력 공간을 정의하는데, 각 클래스에 해당하는 점수(또는 로짓)를 출력합니다. 이러한 점수는 클래스 확률을 계산하는 소프트맥스(softmax) 함수를 통과시킬 수 있습니다.

2. **다중 클래스 분류**: 이미지 분류 작업에서 일반적으로 사용되는 손실 함수는 크로스 엔트로피(cross-entropy)입니다. 크로스 엔트로피 손실 함수는 클래스 확률과 실제 레이블 간의 차이를 계산하기 위해 소프트맥스 함수의 출력을 사용합니다. 따라서 마지막 레이어에는 소프트맥스 함수가 적용되지 않아도 됩니다.

3. **모델의 안정성**: 일부 경우, 마지막 레이어에 활성화 함수를 적용하면 모델의 안정성에 영향을 줄 수 있습니다. 예를 들어, 소프트맥스 함수는 클래스 점수 간의 상대적 크기를 조정하는데, 이로 인해 모델의 출력이 과도하게 크거나 작아지는 문제를 일으킬 수 있습니다. 따라서 클래스 점수를 사용하여 확률을 계산하는 소프트맥스 함수를 적용하지 않고 마지막 레이어를 유지하는 것이 일반적입니다.

요약하자면, Image Classification 모델에서는 마지막 레이어에 활성화 함수를 사용하지 않는 이유는 클래스 확률을 직접 계산하기 위해 선형 레이어를 사용하고, 소프트맥스 함수를 적용하지 않아도 크로스 엔트로피 손실 함수와 함께 사용할 수 있기 때문입니다. 또한, 모델의 안정성을 고려하여 활성화 함수의 부작용을 피하기 위해 마지막 레이어를 유지합니다.

In [22]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        # 레이어들을 nn.ModuleList로 묶음
        self.layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(784, 256),  # 입력 크기: 784, 출력 크기: 256
                nn.ReLU()  # ReLU Activation Function 적용
            ),
            nn.Sequential(
                nn.Linear(256, 64),  # 입력 크기: 256, 출력 크기: 64
                nn.ReLU()  # ReLU Activation Function 적용
            ),
            nn.Linear(64, 10)  # 입력 크기: 64, 출력 크기: 10
        ])

    def forward(self, x: torch.Tensor):
        x = x.flatten(start_dim=1)  # 2D 이미지를 1D 벡터로 펼치기
        for layer in self.layers:
            x = layer(x)  # 각 레이어를 순차적으로 적용
        return x

# MLP 신경망 인스턴스 생성
net = MLP()

# 임의의 입력 데이터 생성
random_input = torch.randn(16, 1, 28, 28)

# MLP 신경망에 입력 데이터 전달하여 출력 얻기
random_output = net(random_input)

# 출력의 크기 출력
print(random_output.shape)


torch.Size([16, 10])


# torch.utils.data.DataLoader 소개

`torch.utils.data.DataLoader`는 PyTorch에서 데이터셋을 미니배치(mini-batch)로 나누어 효율적으로 로딩하기 위한 유틸리티 클래스입니다. `DataLoader`는 데이터셋을 가져와서 병렬적으로 로딩하고 셔플링하며, 사용자가 정의한 변환 및 데이터 처리를 적용할 수 있도록 도와줍니다.

## 주요 기능
1. **데이터 로딩 및 셔플링**: `DataLoader`는 데이터셋을 로딩하고 자동적으로 미니배치로 분할합니다. 또한, 데이터를 셔플하여 학습 과정에서 데이터의 순서에 따른 편향을 줄여줍니다.

2. **병렬 처리**: `DataLoader`는 데이터를 병렬적으로 로딩하여 데이터 로딩 속도를 향상시킵니다. 이를 통해 GPU와의 데이터 처리 병목 현상을 최소화하고, 학습 속도를 향상시킬 수 있습니다.

3. **데이터 변환 및 처리**: `DataLoader`는 사용자가 정의한 변환 함수를 적용하여 데이터를 실시간으로 변환하고 처리할 수 있습니다. 예를 들어, 이미지 데이터에 대해 데이터 증강, 정규화, 임의 변환 등의 작업을 수행할 수 있습니다.

4. **배치 처리**: `DataLoader`는 미니배치 단위로 데이터를 처리하여 효율적인 학습과 추론을 지원합니다. 미니배치 처리는 GPU의 병렬 처리 능력을 최대한 활용하며, 메모리 사용을 최적화합니다.


# 미니배치 (Mini-batch)에 대해

미니배치(Mini-batch)는 데이터를 작은 그룹 단위로 나누어 처리하는 기법입니다. 딥러닝에서는 대량의 데이터를 한 번에 처리하는 것이 어려우므로, 미니배치를 사용하여 데이터를 일부씩 나누어 처리하게 됩니다.

주요 개념과 특징은 다음과 같습니다:

- **데이터 분할**: 미니배치는 전체 데이터셋을 작은 그룹으로 나누어 처리합니다. 이 그룹들은 동일한 크기를 가지며, 각각의 미니배치는 여러 개의 데이터 포인트로 구성됩니다.

- **병렬 처리**: 미니배치는 병렬 처리를 통해 효율적으로 계산할 수 있습니다. 여러 데이터 포인트를 동시에 처리함으로써 GPU와 같은 병렬 컴퓨팅 환경을 최대한 활용하여 학습 속도를 향상시킬 수 있습니다.

- **그래디언트(Gradient) 계산**: 미니배치는 역전파 알고리즘을 사용한 그래디언트 계산에 중요한 역할을 합니다. 미니배치 단위로 손실 함수를 계산하고, 그래디언트를 구한 후 이를 평균 또는 합계하여 전체 데이터셋의 그래디언트를 추정합니다.

- **메모리 효율성**: 미니배치를 사용하면 한 번에 전체 데이터를 메모리에 로드할 필요가 없습니다. 전체 데이터셋이 아닌 작은 미니배치만 메모리에 유지하여 메모리 사용량을 줄이고, 대용량 데이터셋을 처리할 수 있습니다.

미니배치는 모델의 학습과 추론에 널리 사용되는 중요한 개념입니다. 적절한 미니배치 크기를 선택하여 데이터를 효과적으로 처리하고, 학습 속도와 모델의 성능을 향상시킬 수 있습니다.

In [23]:
BATCH_SIZE = 64
# 데이터셋 정의
train_dataset = baseDataset(MNIST_ROOT, True)
val_dataset = baseDataset(MNIST_ROOT, False)

# 데이터로더 정의
train_loader = DataLoader(train_dataset, BATCH_SIZE, True)
val_loader = DataLoader(val_dataset, BATCH_SIZE, True)


# 데이터 로딩 및 확인하는 테스트 코드
for images, labels in train_loader:
    print("Train Batch - Image Shape:", images.shape, "Label Shape:", labels.shape)
    break
    # 이미지와 레이블에 대한 추가적인 작업 수행 가능
    # ...

for images, labels in val_loader:
    print("Validation Batch - Image Shape:", images.shape, "Label Shape:", labels.shape)
    break
    # 이미지와 레이블에 대한 추가적인 작업 수행 가능
    # ...

Train Batch - Image Shape: torch.Size([64, 1, 28, 28]) Label Shape: torch.Size([64])
Validation Batch - Image Shape: torch.Size([64, 1, 28, 28]) Label Shape: torch.Size([64])


In [ ]:
DataLoader()

# 역전파를 이용한 파라미터 업데이트

역전파(Backpropagation)는 인공신경망의 학습 과정에서 손실 함수에 대한 그래디언트(gradient)를 계산하여 파라미터를 업데이트하는 알고리즘입니다. 이 알고리즘은 오차를 역방향으로 전파하여 각 계층의 파라미터에 대한 그래디언트를 계산합니다. 아래는 역전파를 이용한 파라미터 업데이트 과정을 자세히 설명한 것입니다:

1. **순전파**: 입력 데이터를 인공신경망에 주입하여 순전파를 진행합니다. 입력은 여러 계층을 거치며 각 계층은 가중치와 편향에 의해 변환됩니다. 활성화 함수를 통과하여 최종 출력을 얻습니다.

2. **손실 함수 계산**: 순전파를 통해 얻은 출력과 실제 정답(label)을 비교하여 손실 함수를 계산합니다. 손실 함수는 모델의 예측과 실제 값 사이의 차이를 측정하는데 사용됩니다.

3. **역전파**: 손실 함수의 그래디언트(gradient)를 계산하여 각 파라미터에 대한 손실 함수의 미분 값을 얻습니다. 역전파 알고리즘을 통해 각 계층을 거꾸로 통과하면서 그래디언트를 전파합니다.

4. **파라미터 업데이트**: 그래디언트를 사용하여 파라미터를 업데이트합니다. 주로 경사 하강법(gradient descent)을 사용하며, 학습률(learning rate)과 같은 하이퍼파라미터를 설정하여 최적화 과정을 조절할 수 있습니다.

    - 각 파라미터에 대해 다음과 같이 업데이트 수식을 적용합니다:
      ```
      파라미터 = 파라미터 - 학습률 * 그래디언트
      ```

5. **반복**: 순전파, 역전파, 파라미터 업데이트 과정을 반복하여 모델을 학습시킵니다. 이를 여러 번의 에포크(epoch) 동안 반복하여 모델의 성능을 향상시킵니다.

역전파 알고리즘은 그래디언트를 통해 각 파라미터가 손실 함수에 대해 얼마나 기여하는지 계산합니다. 이를 통해 가중치와 편향을 조정하여 손실 함수를 최소화하

고, 모델의 예측을 개선합니다. 학습률은 파라미터 업데이트의 크기를 조절하는 하이퍼파라미터로, 학습 속도와 수렴 속도에 영향을 미칩니다. 적절한 학습률을 선택하여 모델의 최적화 과정을 조절해야 합니다.

In [24]:
EPOCHS = 3  # 총 에포크 수
LR = 1e-2  # 학습률

net = MLP().to(DEVICE)  # MLP 모델 초기화 및 디바이스 설정
criterion = CrossEntropyLoss()  # 손실 함수 정의
optimizer = SGD(net.parameters(), lr=LR)  # 옵티마이저 정의

# Training loop (학습 반복문)
for epoch in range(EPOCHS):
    train_loss = 0.0  # 훈련 손실 초기화
    train_correct = 0  # 훈련 예측 정확도 초기화
    train_total = 0  # 훈련 데이터 총 개수 초기화

    net.train()  # 모델을 훈련 모드로 설정

    # Training phase (훈련 단계)
    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)  # 입력 데이터를 디바이스로 이동
        labels = labels.to(DEVICE)  # 레이블을 디바이스로 이동

        optimizer.zero_grad()  # 그래디언트 초기화

        # Forward pass (순전파)
        outputs = net(inputs)
        loss = criterion(outputs, labels)  # 손실 계산

        # Backward pass and optimization (역전파 및 최적화)
        loss.backward()  # 역전파 수행
        optimizer.step()  # 가중치 업데이트

        # Compute accuracy (정확도 계산)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        train_loss += loss.item()  # 훈련 손실 누적

    train_loss /= len(train_loader)  # 훈련 손실 평균 계산
    train_accuracy = 100 * train_correct / train_total  # 훈련 정확도 계산

    val_loss = 0.0  # 검증 손실 초기화
    val_correct = 0  # 검증 예측 정확도 초기화
    val_total = 0  # 검증 데이터 총 개수 초기화

    net.eval()  # 모델을 평가 모드로 설정

    # Validation phase (검증 단계)
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(DEVICE)  # 입력 데이터를 디바이스로 이동
            labels = labels.to(DEVICE)  # 레이블을 디바이스로 이동

            # Forward pass (순전파)
            outputs = net(inputs)
            loss = criterion(outputs, labels)  # 손실 계산

            # Compute accuracy (정확도 계산)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            val_loss += loss.item()  # 검증 손실 누적

    val_loss /= len(val_loader)  # 검증 손실 평균 계산
    val_accuracy = 100 * val_correct / val_total  # 검증 정확도 계산

    # Print statistics for the current epoch (현재 에포크의 통계 출력)
    print(f"Epoch [{epoch+1}/{EPOCHS}]")
    print(f"  Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"  Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")


Epoch [1/3]
  Train Loss: 1.6793, Train Accuracy: 56.47%
  Val Loss: 0.7731, Val Accuracy: 80.64%
Epoch [2/3]
  Train Loss: 0.5568, Train Accuracy: 85.34%
  Val Loss: 0.4242, Val Accuracy: 88.20%
Epoch [3/3]
  Train Loss: 0.3990, Train Accuracy: 88.88%
  Val Loss: 0.3508, Val Accuracy: 89.98%


# nn.Module의 가중치 저장

`nn.Module`은 파이토치에서 신경망 모델을 구축하는 데 사용되는 기본 클래스입니다. `nn.Module`을 사용하여 정의한 모델은 학습 가능한 파라미터인 가중치(weight)와 편향(bias)을 가지고 있습니다. 이 가중치들은 학습을 통해 업데이트되며 모델의 예측에 사용됩니다.

`nn.Module`의 가중치를 저장하는 방법은 다음과 같습니다:

1. **state_dict() 메서드**: `nn.Module` 객체의 `state_dict()` 메서드를 호출하여 가중치들을 딕셔너리 형태로 가져올 수 있습니다. 이 딕셔너리에는 각 가중치의 이름과 해당 가중치의 값이 저장됩니다.

2. **torch.save() 함수**: `torch.save()` 함수를 사용하여 `state_dict()`로 얻은 가중치 딕셔너리를 파일로 저장할 수 있습니다. 저장된 파일은 나중에 모델을 로드하여 가중치를 복원하는 데 사용될 수 있습니다.

아래는 nn.Module의 가중치를 저장하는 예제입니다:

```python
# 모델의 가중치 저장
torch.save(model.state_dict(), 'model_weights.pth')
```

위 예제에서 `model`은 `nn.Module`로 구성된 모델 객체입니다. `state_dict()` 메서드를 호출하여 모델의 가중치를 딕셔너리 형태로 가져오고, `torch.save()` 함수를 사용하여 이 가중치 딕셔너리를 `'model_weights.pth'`라는 파일에 저장합니다.

이렇게 저장된 가중치는 나중에 `torch.load()` 함수를 사용하여 로드할 수 있으며, 모델에 복원된 가중치를 적용할 수 있습니다. 가중치를 로드하는 방법은 다음과 같습니다:

```python
# 저장된 가중치 로드
model.load_state_dict(torch.load('model_weights.pth'))
```

위 예제에서 `model`은 로드할 모델 객체입니다. `torch.load()` 함수를 사용하여 `'model_weights.pth'` 파일에서 가중치 딕셔너리를 로드하고, `load_state_dict()` 메서드를 사용하여 모델에 가중치를 복원합니다.

가중치를 저장하고 로드함으로써, 학습된 모델을 나중에 다시 사용하거나 전이 학습(transfer learning) 등을 수행할 수 있습니다.

In [ ]:
WEIGHT_PATH = "LinearModel_weights.pth"
torch.save(net.state_dict(), WEIGHT_PATH)